In [74]:
import numpy as np
import pandas as pd
import numbers

# Exercise 2.1

## Item A

In [75]:
def binarizeCategoricalAttributeVector(column):
    categories = set(column)
    categories = list(categories)
    m = np.zeros(shape=(len(column), len(categories)))

    for i in range(len(m)):
        category = categories.index(column[i])
        m[i][category] = 1

    return m

## Item B

In [76]:
def getCategoricalAttributes(df):
    categorical_columns = df.select_dtypes(include=['object'])

    return categorical_columns.columns


## Item C

In [77]:
def readFrameAsMatrix(df):
    categorical_columns = getCategoricalAttributes(df)
    
    for column in categorical_columns:
        replacement = binarizeCategoricalAttributeVector(df[column])
        
        df = df.drop(columns=column)
        df = pd.concat([df, pd.DataFrame(replacement)], axis=1)

    frame = df.to_numpy()    
    return frame

## Item D

In [78]:
dfCredit = pd.read_csv("credits.csv")

npCredit = readFrameAsMatrix(dfCredit)
np.set_printoptions(suppress=True)
print(npCredit)

[[    25. 226802.      7. ...      0.      1.      0.]
 [    38.  89814.      9. ...      0.      1.      0.]
 [    28. 336951.     12. ...      0.      0.      1.]
 ...
 [    58. 151910.      9. ...      0.      1.      0.]
 [    22. 201490.      9. ...      0.      1.      0.]
 [    52. 287927.      9. ...      0.      0.      1.]]


In [79]:
def check_column_conversion(column):
    M = binarizeCategoricalAttributeVector(column)
    vals = list(np.unique(M))
    sorted(vals)
    print("-----------------\nBinarization check\n-----------------")
    print("Dimension check: " + ("OK" if M.shape == (len(column), len(np.unique(column))) else "FAIL"))
    print("Occurring values: " + ("OK" if vals == [0, 1] else "FAIL (there should only be 0s and 1s in the output.)"))
    print("Coherence: " + ("OK" if all(np.sum(M, axis=1) == np.ones(len(column))) else "FAIL (all rows must sum up to 1)"))

def check_category_detection(df, expectedcols):
    print("-----------------\nCheck of category detection\n-----------------")
    act = getCategoricalAttributes(df)
    missing = [c for c in expectedcols if not c in act]
    unexpected = [c for c in act if not c in expectedcols]
    print("Categorical attribute detection: " + ("OK" if len(missing) + len(unexpected) == 0 else "FAIL (undetected columns: " + str(missing) + ", wrongly detected columns: " + str(unexpected) + ")"))
    
def check_frame_conversion(df, num_expected_columns):
    print("-----------------\nConversion check for data frames\n-----------------")
    A = readFrameAsMatrix(df)
    print("Outer Type check: " + ("OK" if type(A) == np.ndarray else "FAIL (not a numpy array but " + str(type(A)) + ")"))
    print("Inner Type check: " + ("OK" if A.dtype in [float, np.float32, np.float64] else "FAIL (dtype of matrix should be something numeric like float and not " + str(A.dtype) + ")"))
    print("Dimensionality check: " + ("OK" if len(A) == len(df) and A.shape[1] == num_expected_columns else "FAIL (expected shape " + str(len(df)) + " x " + str(num_expected_columns) + ", but observed shape " + str(len(A)) + " x " + str(A.shape[1]) + ")"))


## unit test for conversion functions
dfCreditTest = pd.read_csv("credits.csv")
check_column_conversion(dfCreditTest.values[:,1])
check_category_detection(dfCreditTest, ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'class'])
check_frame_conversion(dfCreditTest, 110)

-----------------
Binarization check
-----------------
Dimension check: OK
Occurring values: OK
Coherence: OK
-----------------
Check of category detection
-----------------
Categorical attribute detection: OK
-----------------
Conversion check for data frames
-----------------
Outer Type check: OK
Inner Type check: OK
Dimensionality check: OK


# Exercise 2.2

## Item A

In [80]:
def discretizeBasedOnThresholds(column, thresholds, names=None):
    new_col = column

    if names == None:
        names = []
        for i in range(len(thresholds)+1):
            names.append(f'c{i}')

    for i in range(len(thresholds)):
        for element in range(len(new_col)):
            if new_col[element] not in names:
                if new_col[element] <= thresholds[i]:
                    new_col[element] = names[i]

    for element in range(len(new_col)):
        if new_col[element] not in names:
            new_col[element] = names[len(thresholds)]

    return new_col

## Item B

In [81]:
def discretizeEqualLength(column, k, names = None):
    thresholds = []
    
    min_value = np.min(column)
    max_value = np.max(column)
    w = (max_value - min_value) / k
    
    for i in range(k - 1):
        v = min_value + (i+1) * w
        thresholds.append(v)

    return discretizeBasedOnThresholds(column, thresholds)
    

In [82]:
def discretizeEqualFrequency(column, k, names = None):
    thresholds = []
    
    temp_col = column.copy()
    temp_col.sort()
    
    for i in range(k - 1):
        j = int((i+1) * len(temp_col) / k)
        v = temp_col[j]
        thresholds.append(v)

    return discretizeBasedOnThresholds(column, thresholds)

In [83]:
def test_discretization(column, thresholds, names, expected):
    conv = discretizeBasedOnThresholds(column, thresholds, names)
    print("Conversion test: " + ("OK" if len(conv) == len(expected) and all(conv == expected) else "FAIL (expected \"" + str(expected) +"\" but observed \"" + str(conv) + "\")"))
    
def test_equal_length_discretization(arr, k, expected):
    act = discretizeEqualLength(arr, k)
    print ("Equal Length Discretization: " + ("OK" if all(act == expected) else "FAIL (expected \"" + str(expected) +"\" but observed \"" + str(act) + "\")"))
    
def test_equal_count_discretization(arr, k, expected):
    act = discretizeEqualFrequency(arr, k)
    print ("Equal Count Discretization: " + ("OK" if all(act == expected) else "FAIL (expected \"" + str(expected) +"\" but observed \"" + str(act) + "\")"))

# reproduce results from the lecture
dfIrisTest = pd.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv")
test_discretization(dfIrisTest.values[:,0], [5.2, 6.1, 7], ["very short", "short", "long", "very long"], ["very short", "very short", "very short", "very short", "very short", "short", "very short", "very short", "very short", "very short", "short", "very short", "very short", "very short", "short", "short", "short", "very short", "short", "very short", "short", "very short", "very short", "very short", "very short", "very short", "very short", "very short", "very short", "very short", "very short", "short", "very short", "short", "very short", "very short", "short", "very short", "very short", "very short", "very short", "very short", "very short", "very short", "very short", "very short", "very short", "very short", "short", "very short", "long", "long", "long", "short", "long", "short", "long", "very short", "long", "very short", "very short", "short", "short", "short", "short", "long", "short", "short", "long", "short", "short", "short", "long", "short", "long", "long", "long", "long", "short", "short", "short", "short", "short", "short", "short", "short", "long", "long", "short", "short", "short", "short", "short", "very short", "short", "short", "short", "long", "very short", "short", "long", "short", "very long", "long", "long", "very long", "very short", "very long", "long", "very long", "long", "long", "long", "short", "short", "long", "long", "very long", "very long", "short", "long", "short", "very long", "long", "long", "very long", "long", "short", "long", "very long", "very long", "very long", "long", "long", "short", "very long", "long", "long", "short", "long", "long", "long", "short", "long", "long", "long", "long", "long", "long", "short"])
test_discretization(dfIrisTest.values[:,1], [2.8, 3.6], ["short", "medium", "long"], ["medium", "medium", "medium", "medium", "medium", "long", "medium", "medium", "medium", "medium", "long", "medium", "medium", "medium", "long", "long", "long", "medium", "long", "long", "medium", "long", "medium", "medium", "medium", "medium", "medium", "medium", "medium", "medium", "medium", "medium", "long", "long", "medium", "medium", "medium", "medium", "medium", "medium", "medium", "short", "medium", "medium", "long", "medium", "long", "medium", "long", "medium", "medium", "medium", "medium", "short", "short", "short", "medium", "short", "medium", "short", "short", "medium", "short", "medium", "medium", "medium", "medium", "short", "short", "short", "medium", "short", "short", "short", "medium", "medium", "short", "medium", "medium", "short", "short", "short", "short", "short", "medium", "medium", "medium", "short", "medium", "short", "short", "medium", "short", "short", "short", "medium", "medium", "medium", "short", "short", "medium", "short", "medium", "medium", "medium", "medium", "short", "medium", "short", "medium", "medium", "short", "medium", "short", "short", "medium", "medium", "long", "short", "short", "medium", "short", "short", "short", "medium", "medium", "short", "medium", "short", "medium", "short", "long", "short", "short", "short", "medium", "medium", "medium", "medium", "medium", "medium", "medium", "short", "medium", "medium", "medium", "short", "medium", "medium", "medium"])
test_equal_length_discretization(dfIrisTest.values[:,0], 4, np.array(["c0", "c0", "c0", "c0", "c0", "c1", "c0", "c0", "c0", "c0", "c1", "c0", "c0", "c0", "c1", "c1", "c1", "c0", "c1", "c0", "c1", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c1", "c0", "c1", "c0", "c0", "c1", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c1", "c0", "c2", "c2", "c2", "c1", "c2", "c1", "c2", "c0", "c2", "c0", "c0", "c1", "c1", "c1", "c1", "c2", "c1", "c1", "c2", "c1", "c1", "c1", "c2", "c1", "c2", "c2", "c2", "c2", "c1", "c1", "c1", "c1", "c1", "c1", "c1", "c1", "c2", "c2", "c1", "c1", "c1", "c1", "c1", "c0", "c1", "c1", "c1", "c2", "c0", "c1", "c2", "c1", "c3", "c2", "c2", "c3", "c0", "c3", "c2", "c3", "c2", "c2", "c2", "c1", "c1", "c2", "c2", "c3", "c3", "c1", "c2", "c1", "c3", "c2", "c2", "c3", "c2", "c1", "c2", "c3", "c3", "c3", "c2", "c2", "c1", "c3", "c2", "c2", "c1", "c2", "c2", "c2", "c1", "c2", "c2", "c2", "c2", "c2", "c2", "c1"]))
test_equal_count_discretization(dfIrisTest.values[:,0], 4, np.array(["c0", "c0", "c0", "c0", "c0", "c1", "c0", "c0", "c0", "c0", "c1", "c0", "c0", "c0", "c1", "c1", "c1", "c0", "c1", "c0", "c1", "c0", "c0", "c0", "c0", "c0", "c0", "c1", "c1", "c0", "c0", "c1", "c1", "c1", "c0", "c0", "c1", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c0", "c1", "c0", "c3", "c2", "c3", "c1", "c3", "c1", "c2", "c0", "c3", "c1", "c0", "c2", "c2", "c2", "c1", "c3", "c1", "c1", "c2", "c1", "c2", "c2", "c2", "c2", "c2", "c3", "c3", "c3", "c2", "c1", "c1", "c1", "c1", "c2", "c1", "c2", "c3", "c2", "c1", "c1", "c1", "c2", "c1", "c0", "c1", "c1", "c1", "c2", "c0", "c1", "c2", "c1", "c3", "c2", "c3", "c3", "c0", "c3", "c3", "c3", "c3", "c2", "c3", "c1", "c1", "c2", "c3", "c3", "c3", "c2", "c3", "c1", "c3", "c2", "c3", "c3", "c2", "c2", "c2", "c3", "c3", "c3", "c2", "c2", "c2", "c3", "c2", "c2", "c2", "c3", "c3", "c3", "c1", "c3", "c3", "c3", "c2", "c3", "c2", "c2"]))

Conversion test: OK
Conversion test: OK
Equal Length Discretization: OK
Equal Count Discretization: OK
